In [3]:
import torch, torchvision, torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim 
import numpy as np
import pandas as pd
from skimage import io
import os

In [4]:
import imageio
from skimage.transform import resize
list = pd.read_csv('/Users/michaelcolellajensen/Desktop/Data Sets/annotations.csv')
file_name = list['file_name'].values
print(list.head(1))
for image in file_name:
    #read image
    img_io = imageio.imread('/Users/michaelcolellajensen/Downloads/archive/images/{}'.format(image))

    #Resize image
    img_io = resize(img_io, (26, 26))

    # Write ICO image
    imageio.imwrite('/Users/michaelcolellajensen/Desktop/images/{}'.format(image), img_io)

      file_name  width  height  x1  y1   x2   y2  category
0  000_0001.png    134     128  19   7  120  117         0


In [5]:
#build a new calss object for the images 
class TrafficSigns(Dataset):
    #constructure will need csv file of labels images and the transform function defined above
    def __init__(self, 
                 csv_file, 
                 root_directory, 
                 transform = None):
        self.labels = pd.read_csv(csv_file)
        self.root_directory = root_directory
        self.transform = transform
    #returns the length 
    def __len__(self):
        return len(self.labels)
    #get data index by indes
    def __getitem__(self, i):
        image_path = os.path.join(self.root_directory, self.labels.iloc[i,0])
        image = io.imread(image_path)
        y_label = torch.tensor(int(self.labels.iloc[i, 1]))
        
        #if statement needed since transform can be set to None 
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label)

In [6]:
#hyperprams
learning_rate = 5e-4
#3 for RGB values 
in_channel = 3
#classes from data set 
num_classes = 57
# arbitray choice 
batch_size = 24
#total number of epochs used to train the model 
epochs = 3

traffic_dataset = TrafficSigns('/Users/michaelcolellajensen/Desktop/Data Sets/annotations.csv',
                       '/Users/michaelcolellajensen/Desktop/images',
                       transform = transforms.ToTensor())
train_size = int(0.8 * len(traffic_dataset))
test_size = len(traffic_dataset) - train_size
train, test = torch.utils.data.random_split(traffic_dataset,
                                            [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train,
                                           batch_size= batch_size, 
                                           shuffle= True, 
                                           num_workers= 6)

test_loader = torch.utils.data.DataLoader(test, 
                                          batch_size = batch_size, 
                                          shuffle= True, 
                                          num_workers= 6)

In [7]:
#Create a fully connected nn
class Net(nn.Module):
    #use the constructor w/ arguements size of data and number of classes
    def __init__(self, 
                 input_size, 
                 num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    #define your forward step function with relu as the non-linear function of the weights
    #x will be the datapassed to the model 
    def forward(self, x):
        x=f.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
#instantiate the class object of NN
net = Net(2028, 57)
learning_rate = 5e-4
criterion = nn.CrossEntropyLoss()
nn_optimizer = optim.Adam(net.parameters(), 
                          lr = learning_rate)

#train on multiple epochs using the criterion and gradient decent algorthim estabilished above
for epoch in range(1):
    for i, (data,target) in enumerate(train_loader):
     data = data.reshape(data.shape[0], -1)
     nn_optimizer.zero_grad()
     #forward
     outputs = net(data)
     loss = criterion(outputs, target)
     #backward propigation
     nn_optimizer.zero_grad()
     loss.backward()
     nn_optimizer.step()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'TrafficSigns' on <module '__main__' (built-in)>


In [ ]:
#evaluate model performance on the test data
#create variables to count correct predictins and total predictions
nn_correct, nn_total = 0, 0
#empty list to hold the predicted probabilities of each class 
nn_predictions = []
#model must be put in eval mode before testing 
net.eval()
for i, data in enumerate(test, 0):
    inputs, labels = data
    inputs = inputs.view(-1, 15750) 
    outputs = net(inputs) 
    _, predicted = torch.max(outputs.data, 1)
    nn_predictions.append(outputs)
    nn_total += labels.size(0)
    nn_correct += (predicted == labels).sum()
    nn_percent = format((100*nn_correct/nn_total), '.2f')
print('The test accuracy of the model is: {}%'.format(nn_percent))

In [ ]:
#building a convolutional neural network for images 
class CNNet(nn.Module):
    def __init__(self,num_classes):
        super(CNNet, self).__init__()
        #using a 3x3 kernal and 1x1 padding/stride the num of inut features will match output features
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=15,
                               kernel_size= (3,3),
                               stride=(1,1),
                               padding= (1,1),
                               bias=True)
        #in channels on second layer must match out on first layer 
        self.conv2 = nn.Conv2d(in_channels= 15,
                               out_channels= 30,
                               kernel_size=(3,3),
                               stride = (1,1),
                               padding=(1,1),
                               bias=True)
        #can use max pooling again but careful to ensure features are still whole numbers 
        self.pool = nn.MaxPool2d(2,2)
        #last level is fully connected with num out channels * original image dem /2 and /2 again
        self.fc = nn.Linear(30 * 13 * 13 * 3, num_classes)
        
    def forward(self, x):
            x = f.relu(self.conv1(x))
            x = f.relu(self.conv2(x))
            x = self.pool(x)
            x = x.reshape(x.shape[0], -1)
            x = self.fc(x)
            return x

In [ ]:
#instantiate the class object of CNNet
cnn = CNNet(57)
cnn_optimizer = optim.Adam(cnn.parameters(), lr = learning_rate)
#training the CNN 

for epoch in range(1):
    for i, (data,target) in enumerate(train_loader):
     cnn_optimizer.zero_grad()
     #forward
     outputs = cnn(data)
     loss = criterion(outputs, target)
     #backward propigation
     cnn_optimizer.zero_grad()
     loss.backward()
     cnn_optimizer.step()

In [ ]:
#evaluate the CNN performace 
cnn.eval()
cnn_correct, cnn_total = 0, 0
#empty list to hold the predicted probabilities of each class 
cnn_predictions = []
for i, data in enumerate(test_loader, 0):
    inputs, labels = data
    outputs = cnn(inputs) 
    _, predicted = torch.max(outputs.data, 1)
    cnn_predictions.append(predicted)
    cnn_total += labels.size(0)
    cnn_correct += (predicted == labels).sum()
    cnn_percent = format((cnn_correct/cnn_total * 100), '.2f')
print('The test accuracy of the model is: {}%'.format(cnn_percent))